<a href="https://colab.research.google.com/github/sudeshna0501/twitter-sentiment-analysis/blob/main/BERT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/My Drive/Tweets.csv')

In [ ]:
X= df['text']
Y= df['airline_sentiment']

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import np_utils

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(df['airline_sentiment'])

In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [ ]:
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

In [ ]:
X = X.apply(clean_doc)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = \
train_test_split(X, dummy_y,test_size=0.2)

In [ ]:
def joining(ele):
 return " ".join(str(v) for v in ele)


In [ ]:
snt_emb = X_train.apply(joining)

In [ ]:
snt_emb = snt_emb.tolist()

In [ ]:
!pip install sentence-transformers --quiet

     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 4.4 MB 55.4 MB/s 
     |████████████████████████████████| 1.2 MB 67.4 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
     |████████████████████████████████| 6.6 MB 64.8 MB/s 


In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
sentence_embeddings = sbert_model.encode(snt_emb)

In [ ]:
snt_emb_test = X_test.apply(joining)

In [ ]:
snt_emb_test = snt_emb_test.tolist()

In [ ]:
testing_embeddings = sbert_model.encode(snt_emb_test)

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate


In [ ]:
# define the model
def define_model(bert_len):
  inputs4 =Input(shape=(bert_len,))
  denseb1=Dense(300, activation='tanh')(inputs4)
  dense1 = Dense(300, activation='relu')(denseb1)
  dense2 = Dense(100, activation='relu')(dense1)
  dense3 = Dense(20, activation='relu')(dense2)
  dense4 = Dense(10, activation='relu')(dense3)
  outputs = Dense(3, activation='softmax')(dense4)

  model = Model(inputs=[inputs4], outputs=outputs)
  # compile
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  # summarize
  print(model.summary())
  plot_model(model, show_shapes=True, to_file='multichannel.png')
  return model


In [ ]:
# define model
model = define_model(sentence_embeddings.shape[1])
print(model.summary())

#trainX_lstm,trainX_cnn,trainX_cnn,

# fit model
model.fit([sentence_embeddings], y_train, epochs=50, batch_size=20)
# save the model
model.save('model.h5')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 768)]             0         
                                                                 
 dense_6 (Dense)             (None, 300)               230700    
                                                                 
 dense_7 (Dense)             (None, 300)               90300     
                                                                 
 dense_8 (Dense)             (None, 100)               30100     
                                                                 
 dense_9 (Dense)             (None, 20)                2020      
                                                                 
 dense_10 (Dense)            (None, 10)                210       
                                                                 
 dense_11 (Dense)            (None, 3)                 33  

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.models import Sequential

In [ ]:
model = load_model('model.h5')
# evaluate model on training dataset
loss, acc = model.evaluate([sentence_embeddings], y_train, verbose=0)
print('Train Accuracy: %f' % (acc*100))


# evaluate model on test dataset dataset
loss, acc = model.evaluate([testing_embeddings], y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

predictions = model.predict(testing_embeddings)

Train Accuracy: 97.908127
Test Accuracy: 77.459013
